In [11]:
import pandas as pd
import glob
import plotly.graph_objects as go
import numpy as np

# %%
files_metrics = glob.glob('../results_data/multiple_seed/metrics/*')
files_glo_podo = glob.glob('../results_data/multiple_seed/glo_podo/*')
labels = ['pix2pix', 'pix2pix L1', 'pix2pix L1+reset', 'pix2pix patches', 'cycleGAN', 'cycleGAN patches']
file_names = ['test_pix2pix_PAS2IF_van_patches', 'test_pix2pix_PAS2IF_L1_patches', 'test_pix2pix_PAS2IF_L1_reset_patches', 'test_pix2pix_PAS2IF_4_patches', 'test_cycle_PAS2IF_van_patches', 'test_cycle_PAS2IF_4_patches']
metrics = ['SSIM', 'MAE', 'MSE', 'FID', 'Dice podo. object', 'Dice podo. pixel', 'Dice glom. pixel']

In [12]:
# Read Multi Seeds
df_final_podo_glo = pd.DataFrame()
for file in files_glo_podo:
    df = pd.DataFrame()
    name = file.split('/')[4].split('_seed')[0]
    seed = file.split('/')[4].split('_seed_')[1].split('_')[0]
    epoche =  file.split('/')[4].split('_epoch')[1].split('_')[0]
    df_temp = pd.read_excel(file)
    df_temp = df_temp[['Network dice object', 'Network dice pixel']]
    cell_type = 'glom.' if 'glom' in file else 'podo.'
    df['value'] = df_temp['Network dice object'].append(df_temp['Network dice pixel'])
    df['name'] = name
    df['metric'] = np.concatenate([np.repeat(f'Dice {cell_type} object',len(df_temp)),
                       np.repeat(f'Dice {cell_type} pixel', len(df_temp))])
    df['seed'] = seed
    df['epoch'] = epoche
    df_final_podo_glo = pd.concat([df_final_podo_glo, df[['name', 'metric', 'value','seed', 'epoch']]])


In [14]:
# Mean all seeds
for label, file_name in zip(labels, file_names):
    df_final_podo_glo = df_final_podo_glo.replace({file_name: label})
df_final_podo_glo = df_final_podo_glo.reset_index()
df_final_podo_glo = df_final_podo_glo.groupby(['name', 'metric', 'index']).agg('mean').reset_index()
df_final_podo_glo = df_final_podo_glo.drop(columns=['index'])
df_final_podo_glo

,name,metric,value
0,cycleGAN,Dice glom. object,1.000000
1,cycleGAN,Dice glom. object,1.000000
2,cycleGAN,Dice glom. object,1.000000
3,cycleGAN,Dice glom. object,1.000000
4,cycleGAN,Dice glom. object,1.000000
...,...,...,...
307,pix2pix patches,Dice podo. pixel,0.294817
308,pix2pix patches,Dice podo. pixel,0.447625
309,pix2pix patches,Dice podo. pixel,0.254961
310,pix2pix patches,Dice podo. pixel,0.414285


In [15]:
df_final_metric = pd.DataFrame()
for file in files_metrics:
    df = pd.DataFrame()
    name = file.split('/')[4].split(' ')[0]
    metric = file.split(' ')[1].split('.')[0]
    df = pd.read_csv(file)
    df = df.filter(regex=r' (?!.*?(MIN|MAX)).*', axis=1)
    df['value'] = df.mean(axis=1)
    df['metric'] = metric
    df['name'] = name
    df_final_metric = pd.concat([df_final_metric, df[['name', 'metric', 'value']]])

for label, file_name in zip(labels, file_names):
    df_final_metric = df_final_metric.replace({file_name: label})
df_final_metric

,name,metric,value
0,cycleGAN,FID,246.813554
1,cycleGAN,FID,210.859351
2,cycleGAN,FID,210.387416
3,cycleGAN,FID,188.766631
4,cycleGAN,FID,250.105527
...,...,...,...
47,pix2pix L1,MAE,0.030937
48,pix2pix L1,MAE,0.033692
49,pix2pix L1,MAE,0.039708
50,pix2pix L1,MAE,0.032256


In [16]:
df_final = pd.concat([df_final_metric, df_final_podo_glo[['name', 'metric', 'value']]])
df_final

,name,metric,value
0,cycleGAN,FID,246.813554
1,cycleGAN,FID,210.859351
2,cycleGAN,FID,210.387416
3,cycleGAN,FID,188.766631
4,cycleGAN,FID,250.105527
...,...,...,...
307,pix2pix patches,Dice podo. pixel,0.294817
308,pix2pix patches,Dice podo. pixel,0.447625
309,pix2pix patches,Dice podo. pixel,0.254961
310,pix2pix patches,Dice podo. pixel,0.414285


In [17]:
def boxplot(df: pd.DataFrame, metric: str, labels:list, export=False) -> None:
    df_plot = df[df.metric == metric]
    # colors = ['#4038E8', '#<3A63F2', '#408ADB', '#3AC4F2', '#30F541', '#3EF595']
    colors = ['#ED7D1F', '#ED7D1F', '#ED7D1F', '#ED7D1F', '#33A125', '#33A125']
    fig = go.Figure()
    for col, name in zip(colors, labels):
        fig.add_trace(go.Box(y=df_plot['value'][df_plot.name == name],
            name=name, marker_color=col, boxmean=True, showlegend=False, boxpoints='all'
        ))
    if metric == 'SSIM':
        fig.update_layout(yaxis_range=[0.2,1])
    fig.update_xaxes(categoryorder='array', categoryarray=labels)
    fig.update_layout(template='simple_white',
                      font_family="Computer Modern",
                      font_size=20,
                      yaxis_title=metric,
                      boxgap=0.3,
                      height=500,
                      width=1000,
                      margin=dict(l=50, r=20, t=20, b=20)
                      )
    fig.show()

    if export:
        fig.write_image(f'../graphics/{metric}.pdf')

for metric in metrics:
    boxplot(df_final, metric, labels=labels, export=True)



In [ ]:
## Read Channel data
files = glob.glob('../results_data/channels/*')

df_channels = pd.DataFrame()
for file in files:
        name = file.split('/')[3].split(' ')[0]
        metric = file.split(' ')[3].split('.')[0]
        df = pd.read_csv(file)
        df = df.filter(regex=r' (?!.*?(MIN|MAX)).*', axis=1)
        df['value'] = df.iloc[:, 0]
        df['metric'] = metric
        df['name'] = name
        df_channels = pd.concat([df_channels, df[['name', 'metric', 'value']]])

metrics_channels = ['SSIM', '0', '1', '2']
label_channels = ['all', 'DAPI', 'WT1', 'DACH1']
for label, metric in zip(label_channels, metrics_channels):
    df_channels = df_channels.replace({metric: label})

df_channels.name.unique()## Read Channel data

In [4]:
## Boxplot for Channels

def boxplot_channels(df: pd.DataFrame, name:str, export=False) -> None:
    df = df[df.name == name]
    colors = ['#808080', '#DF2F27', '#31DF41', '#334AF9']
    fig = go.Figure()
    for col, metric in zip(colors, label_channels):
        fig.add_trace(go.Box(y=df['value'][df.metric == metric],
            name=metric, marker_color=col, boxmean=True, showlegend=False, boxpoints='all'
        ))
    fig.update_xaxes(categoryorder='array')
    fig.update_layout(template='simple_white',
                      font_family="Computer Modern",
                      font_size=20,
                      yaxis_title='SSIM',
                      yaxis_range=[0.2,1],
                      # boxgap=0.3,
                      height=500,
                      width=900,
                      margin=dict(l=50, r=20, t=20, b=20)
                      )
    fig.show()

    if export:
        fig.write_image(f'../graphics/channels_{name}.pdf')

boxplot_channels(df_channels, name='test_cycle_PAS2IF_van_patches_seed_975', export=True)
boxplot_channels(df_channels, name='test_pix2pix_PAS2IF_L1_patches', export=True)
boxplot_channels(df_channels, name='test_pix2pix_PAS2IF_4_patches', export=True)